In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
info = pd.read_csv('../input/ibm-hr-analytics-attrition-dataset/WA_Fn-UseC_-HR-Employee-Attrition.csv')

In [ ]:
info

but, some columns here seem quite useless, like "EmployeeNumber" (perhaps order they were hired? database is unclear) and "EmployeeCount" (all 1s) and "Over18" (all yes) and "StandardHours" (all 80), so they'll just be removed now

In [ ]:
#perc =[.20, .40, .60, .80] 
#desc = info.describe(percentiles = perc, include = ['float', 'int']) 
#desc

In [ ]:
#import matplotlib.pyplot as plt
#for i in info.columns:
#    plt.figure()
#    plt.hist(info[i])
#    plt.suptitle(i, fontsize=20)


In [ ]:
info.drop(['EmployeeNumber','EmployeeCount','Over18','StandardHours'], inplace=True,axis=1)

now we OHE the categorical variables, including likert scale type variables, word-categorical ones first

In [ ]:
columns = info.columns
num_columns = info._get_numeric_data().columns
catlist = list(set(columns)-set(num_columns))

In [ ]:
catlist

In [ ]:
catlist.remove('Attrition')

In [ ]:
tempcat = pd.concat([pd.get_dummies(info[col],col) for col in catlist], axis=1)
print(tempcat.columns)

In [ ]:
tempcat

and thats the OHE categorical. now lets do the likert scale ones, these ones i'll just manually do because it tells us all of the necessary info already.

In [ ]:
liklist = ['Education','EnvironmentSatisfaction','JobInvolvement','JobSatisfaction','PerformanceRating','RelationshipSatisfaction','WorkLifeBalance','JobLevel','StockOptionLevel']
templik = pd.concat([pd.get_dummies(info[col],col) for col in liklist], axis=1)
print(templik.columns)

In [ ]:
templik

odd, looks like we're missing 1 and 2 for PerformanceRating. But maybe this is not a bug? lets test

In [ ]:
info['PerformanceRating'].plot.hist()

Ok, indeed, it's not a bug. cool, let's keep going. Let's first purge the original columns in liklist and catlist from info dataframe before concatenating them all together.

In [ ]:
info.drop(liklist,axis=1,inplace=True)

In [ ]:
info.drop(catlist,axis=1,inplace=True)

In [ ]:
newinfo = pd.concat([info['Attrition'],tempcat,templik],axis=1)

In [ ]:
info.drop('Attrition', axis=1, inplace=True)

In [ ]:
info

In [ ]:
info1 = pd.DataFrame(columns=info.columns)
for i in info.columns:
    info1[i] = pd.qcut(info[i], 5, labels=False,duplicates='drop')

In [ ]:
info1

In [ ]:
newinfo = pd.concat([info1, newinfo],axis=1)

In [ ]:
newinfo

all thats left before we can start training etc is turning attrition into a 0 or 1 target.

In [ ]:
newinfo['Attrition'].replace({"Yes":1,"No":0}, inplace=True)

In [ ]:
newinfo

In [ ]:
x = newinfo.drop(['Attrition'],axis=1)

In [ ]:
y = newinfo['Attrition']

In [ ]:
x

In [ ]:
y

test train split time

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2)

20% of data will be kept as metric to evaluate model

so we'll try two models, a logistic regression and a shallow neural net

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression().fit(x_train,y_train)

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
score = accuracy_score(y_test,y_pred)
print(classification_report(y_test,y_pred))

In [ ]:
score

In [ ]:
y_probpred = model.predict_proba(x_test)

In [ ]:
probs = []

In [ ]:
for term in y_probpred:
    probs.append(term[1])

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,probs)

cool, so not too bad. but lets identify the key features.

In [ ]:
from sklearn.feature_selection import RFE
selector = RFE(LogisticRegression(), n_features_to_select = 5)

In [ ]:
selector = selector.fit(x_train,y_train)

In [ ]:
y_predRFE = selector.predict(x_test)

In [ ]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
score = accuracy_score(y_test,y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)


In [ ]:
print("general accuracy" + str(score))
print("precision" + str(precision))
print("recall" + str(recall))


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

In [ ]:
print ('coefficients',selector.estimator_.coef_)

In [ ]:
selector.support_

In [ ]:
x_train.columns[[15,29,47,51,69]]

In [ ]:
x_train

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
nn = keras.Sequential()
nn.add(keras.Input(78))
nn.add(layers.Dense(20, activation="relu"))
nn.add(layers.Dense(20, activation="relu"))
nn.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
nn.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [ ]:
nn.fit(x_train,y_train,epochs=4,validation_data=(x_test,y_test))

In [ ]:
nn.fit(x_train,y_train,epochs=4,validation_data=(x_test,y_test))

In [ ]:
nn.fit(x_train,y_train,epochs=4,validation_data=(x_test,y_test))

In [ ]:
predictions = nn.predict(x_test)

In [ ]:
predictions = predictions.flatten()

In [ ]:
newlist = []
for term in predictions:
    if term > 0.5:
        newlist.append(1)
    else:
        newlist.append(0)


In [ ]:
newlist

In [ ]:
score = accuracy_score(y_test,newlist)
precision = precision_score(y_test, newlist)
recall = recall_score(y_test, newlist)

In [ ]:
print("general accuracy" + str(score))
print("precision" + str(precision))
print("recall" + str(recall))

In [ ]:
print(classification_report(y_test,newlist))

In [ ]:
info2 = pd.read_csv('../input/ibm-hr-analytics-attrition-dataset/WA_Fn-UseC_-HR-Employee-Attrition.csv')

In [ ]:
import matplotlib.pyplot as plt
for i in info2.columns:
    plt.figure()
    plt.hist(info2[i])
    plt.suptitle(i, fontsize=20)


In [ ]:
import statsmodels.api as sm
data = info2[['JobRole','Attrition']]
print(sm.stats.Table.from_data(data).table_orig)
data = info2[['OverTime','Attrition']]
print(sm.stats.Table.from_data(data).table_orig)
data = info2[['JobLevel','Attrition']]
print(sm.stats.Table.from_data(data).table_orig)
data = info2[['EnvironmentSatisfaction','Attrition']]
print(sm.stats.Table.from_data(data).table_orig)
data = info2[['JobInvolvement','Attrition']]
print(sm.stats.Table.from_data(data).table_orig)

143/443

In [ ]:
150/882